In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt

In [2]:
myDataset = pd.read_csv("Health_Risk_Prediction.csv")

In [3]:
myDataset

,age,weight,height,exercise,sleep,sugar_intake,smoking,alcohol,married,profession,bmi,health_risk
0,56,67,195,low,6.1,medium,yes,yes,yes,office_worker,17.6,high
1,69,76,170,high,6.9,high,no,no,no,teacher,26.3,high
2,46,106,153,high,6.6,low,yes,no,no,artist,45.3,high
3,32,54,186,medium,8.5,medium,no,no,no,artist,15.6,low
4,60,98,195,high,8.0,low,no,no,yes,teacher,25.8,high
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,42,75,187,medium,7.2,low,no,no,yes,driver,21.4,low
4996,39,86,183,low,7.4,low,no,no,yes,engineer,25.7,high
4997,48,56,194,none,7.2,high,yes,yes,yes,engineer,14.9,high
4998,34,109,148,medium,9.9,medium,no,no,yes,doctor,49.8,low


In [4]:
myDataset.isnull().sum()


age             0
weight          0
height          0
exercise        0
sleep           0
sugar_intake    0
smoking         0
alcohol         0
married         0
profession      0
bmi             0
health_risk     0
dtype: int64

In [5]:
def quanqual(myDataset):
    quan=[]
    qual=[]
    for ColumnName in myDataset.columns:
           # print(ColumnName)
            if (myDataset[ColumnName].dtype=='O'):
#print("it is quantitative variable / column")   #numbers may be discrete or continous
                qual.append(ColumnName)
            else:
               # print("it is qualitative variable / column")      # Categorical data - nominal, or ordinal
                quan.append(ColumnName)
    return quan,qual

In [6]:
quan_col,qual_col = quanqual(myDataset)

In [7]:
Quan = myDataset[quan_col]

In [8]:
Qual = myDataset[qual_col]

In [9]:
#myDatasetNew = myDataset

In [11]:
#myDatasetNew
#myDatasetNew['health_risk'] = myDatasetNew['health_risk'].replace({
 #   'low': 1,
  #  'medium': 2,
   # 'high': 3
#})

C:\Users\USER\AppData\Local\Temp\ipykernel_4896\408404260.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  myDatasetNew['health_risk'] = myDatasetNew['health_risk'].replace({


In [1]:
#Data processing 
#One hot encoding for Qualitative data (input Varaible)
#Label encoding for Output varaible

In [9]:
df = myDataset.copy() 

In [10]:
onehot_cols = ['profession','smoking','alcohol','married','health_risk']
df_onehot = pd.get_dummies(df[onehot_cols], drop_first=True)

# Skip this step. Convert all to Label Encoding and check the Accuracy

In [11]:
#df_onehot
df_onehot.replace({True: 1, False: 0}, inplace=True)
# Skip this step. Convert all to Label Encoding and check the Accuracy

C:\Users\USER\AppData\Local\Temp\ipykernel_20152\3402418838.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_onehot.replace({True: 1, False: 0}, inplace=True)


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#label_cols = ['exercise', 'sugar_intake']
label_columns = ['exercise', 'sugar_intake','profession','smoking','alcohol','married','health_risk']

#df_label = df[label_cols].apply(le.fit_transform)
df_label = df[label_columns].apply(le.fit_transform)

In [11]:
df_numeric = df.select_dtypes(include=['int64', 'float64'])

In [62]:
#Merge all the data into myNewDataset
#df_onehot,df_label,df_numeric

In [12]:
#myNewDataset = pd.concat([df_numeric, df_label,df_onehot], axis=1)
newdata = pd.concat([df_numeric, df_label,], axis=1)

In [15]:
myNewDataset.head()

,age,weight,height,sleep,bmi,exercise,sugar_intake,profession_doctor,profession_driver,profession_engineer,profession_farmer,profession_office_worker,profession_student,profession_teacher,smoking_yes,alcohol_yes,married_yes,health_risk_low
0,56,67,195,6.1,17.6,1,2,0,0,0,0,1,0,0,1,1,1,0
1,69,76,170,6.9,26.3,0,0,0,0,0,0,0,0,1,0,0,0,0
2,46,106,153,6.6,45.3,0,1,0,0,0,0,0,0,0,1,0,0,0
3,32,54,186,8.5,15.6,2,2,0,0,0,0,0,0,0,0,0,0,1
4,60,98,195,8.0,25.8,0,1,0,0,0,0,0,0,1,0,0,1,0


In [49]:
newdata.drop(['height', 'weight'], axis=1, inplace=True)

In [50]:
newdata.head()

,age,sleep,bmi,exercise,sugar_intake,profession,smoking,alcohol,married,health_risk
0,56,6.1,17.6,1,2,5,1,1,1,0
1,69,6.9,26.3,0,0,7,0,0,0,0
2,46,6.6,45.3,0,1,0,1,0,0,0
3,32,8.5,15.6,2,2,0,0,0,0,1
4,60,8.0,25.8,0,1,7,0,0,1,0


In [51]:
newdata.tail()

,age,sleep,bmi,exercise,sugar_intake,profession,smoking,alcohol,married,health_risk
4995,42,7.2,21.4,2,1,2,0,0,1,1
4996,39,7.4,25.7,1,1,3,0,0,1,0
4997,48,7.2,14.9,3,0,3,1,1,1,0
4998,34,9.9,49.8,2,2,1,0,0,1,1
4999,72,6.3,21.2,0,2,2,0,0,1,1


In [17]:
myNewDataset.shape

(5000, 18)

In [18]:
myNewDataset.notnull().sum()

age                         5000
weight                      5000
height                      5000
sleep                       5000
bmi                         5000
exercise                    5000
sugar_intake                5000
profession_doctor           5000
profession_driver           5000
profession_engineer         5000
profession_farmer           5000
profession_office_worker    5000
profession_student          5000
profession_teacher          5000
smoking_yes                 5000
alcohol_yes                 5000
married_yes                 5000
health_risk_low             5000
dtype: int64

In [52]:
#myNewDataset
newdata

,age,sleep,bmi,exercise,sugar_intake,profession,smoking,alcohol,married,health_risk
0,56,6.1,17.6,1,2,5,1,1,1,0
1,69,6.9,26.3,0,0,7,0,0,0,0
2,46,6.6,45.3,0,1,0,1,0,0,0
3,32,8.5,15.6,2,2,0,0,0,0,1
4,60,8.0,25.8,0,1,7,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4995,42,7.2,21.4,2,1,2,0,0,1,1
4996,39,7.4,25.7,1,1,3,0,0,1,0
4997,48,7.2,14.9,3,0,3,1,1,1,0
4998,34,9.9,49.8,2,2,1,0,0,1,1


In [53]:
indep_X=newdata.drop('health_risk', axis=1)
dep_Y=newdata['health_risk']

In [54]:
indep_X.columns

Index(['age', 'sleep', 'bmi', 'exercise', 'sugar_intake', 'profession',
       'smoking', 'alcohol', 'married'],
      dtype='object')

In [55]:
def selectkbest(indep_X,dep_Y,n):
        test = SelectKBest(score_func=chi2, k=n)
        fit1= test.fit(indep_X,dep_Y)
        selectk_features = fit1.transform(indep_X)
        return selectk_features
    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.30, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
 
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
   
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm

In [56]:
def selectk_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Logistic','SVMl','SVMnl','KNN','Navie','Decision','Random'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
        dataframe['SVMl'][idex]=accsvml[number]
        dataframe['SVMnl'][idex]=accsvmnl[number]
        dataframe['KNN'][idex]=accknn[number]
        dataframe['Navie'][idex]=accnav[number]
        dataframe['Decision'][idex]=accdes[number]
        dataframe['Random'][idex]=accrf[number]
    return dataframe

In [70]:
kbest=selectkbest(indep_X,dep_Y,5)       

acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [71]:
kbest

array([[56. , 17.6,  2. ,  1. ,  1. ],
       [69. , 26.3,  0. ,  0. ,  0. ],
       [46. , 45.3,  1. ,  1. ,  0. ],
       ...,
       [48. , 14.9,  0. ,  1. ,  1. ],
       [34. , 49.8,  2. ,  0. ,  0. ],
       [72. , 21.2,  2. ,  0. ,  0. ]])

In [72]:
X_train, X_test, y_train, y_test=split_scalar(kbest,dep_Y)
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train,y_train,X_test)
acclog.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=svm_linear(X_train,y_train,X_test)  
accsvml.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
accsvmnl.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=knn(X_train,y_train,X_test)  
accknn.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Navie(X_train,y_train,X_test)  
accnav.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=Decision(X_train,y_train,X_test)  
accdes.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=random(X_train,y_train,X_test)  
accrf.append(Accuracy)

In [73]:
result=selectk_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf)


C:\Users\USER\AppData\Local\Temp\ipykernel_20672\2634354880.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe['Logistic'][idex]=acclog[number]
C:\Users\USER\AppData\Local\Temp\ipykernel_20672\2634354880.py:6: FutureWarning: Chain

In [35]:
#k=10
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.84,0.84,0.88,0.907333,0.832667,0.991333,0.978667


In [60]:
#k=10 - Label Encoding, dropped height and weight
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.824,0.822667,0.841333,0.848667,0.822,0.86,0.856667


In [31]:
#k=12
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.841333,0.839333,0.866667,0.867333,0.833333,0.992667,0.969333


In [39]:
#k=9
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.838667,0.838667,0.892,0.924,0.834,0.992,0.988667


In [64]:
#K=9 Label Encoding
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.837333,0.843333,0.886667,0.892667,0.818667,0.992667,0.975333


In [44]:
#k=7
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.824,0.822667,0.841333,0.848667,0.822,0.86,0.856667


In [69]:
#K= 7 Label Encoding
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.839333,0.840667,0.907333,0.945333,0.821333,0.993333,0.988


In [48]:
#k15
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.842,0.838,0.865333,0.832,0.830667,0.993333,0.953333


In [74]:
#K=5
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
ChiSquare,0.827333,0.828667,0.85,0.844,0.807333,0.851333,0.857333


In [75]:
kbest1 = SelectKBest(score_func=chi2, k=7)
kbest1.fit(indep_X, dep_Y)
mask = kbest1.get_support()


In [53]:
selected_features = indep_X.columns[mask]
print(selected_features)

Index(['age', 'weight', 'height', 'sleep', 'bmi', 'exercise', 'sugar_intake',
       'profession_doctor', 'profession_driver', 'profession_engineer',
       'profession_farmer', 'profession_teacher', 'smoking_yes', 'alcohol_yes',
       'married_yes'],
      dtype='object')


In [76]:
selected_features = indep_X.columns[mask]
print(selected_features)

Index(['age', 'sleep', 'bmi', 'exercise', 'sugar_intake', 'smoking',
       'alcohol'],
      dtype='object')


In [ ]:
# Kselect best with 15 columns for Decesion Tree Algorithm. with Label and One hot encoding
# K select with 12 columns best for decesion tree, using Only Label Encoding
